In [1]:
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import time
from rouge import Rouge 
from multiprocessing import Pool

In [15]:
d1 = pd.read_csv("Eval2_Query_Embeddings.csv",sep=',',header=None)
d2 = pd.read_csv("Eval2_Response_Embeddings.csv",sep=',',header=None)

In [16]:
d1.shape, d2.shape

((415380, 30), (415380, 30))

In [17]:
d = pd.concat([d1,d2],axis=1)

In [18]:
d.to_csv("Eval2_Embeddings.csv",sep=',',header=None)

In [26]:
l = ""
for i in range(0,len(d),10):
    l+=d['query'][i]+" "
    for j in range(i,i+10):
        l+=d['response'][j]+" "

In [29]:
file = open("text_doc.txt","w")
file.write(l)
file.close()

In [6]:
df = pd.read_csv("Clean_data_latest.tsv", sep='\t', header=None)
df = df.drop(index=df[df.isnull().any(axis=1)].index).reset_index(drop=True)

In [7]:
df.to_csv("Clean_data_latest.tsv", sep='\t', header=None, index=None)
df.to_csv("Clean_data_latest.csv", sep=',', index=None)

In [2]:
data = pd.read_csv("fully_processed_data.csv").iloc[:200000,:]

In [3]:
data['query'] = data['query'].astype(str)
data['response'] = data['response'].astype(str)

In [4]:
def get_cosine_sim(data,i):
    if(i%25 == 0):
        print(i)
    sent = (data['query'][i], *(data['response'][i:i+10].T))
    text = [t for t in sent]
    vectorizer = TfidfVectorizer()
    vect = vectorizer.fit_transform(text).toarray()
    vectors = [t for t in vect]
    return cosine_similarity(vectors)[0,1:].T

In [22]:
def jaccard_sim(data, i, stime): 
    if(i%25 == 0):
        print(i)
        print(time.time()-stime)
    a = set(data['query'][i].split())
    b = set(data['response'][i].split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [6]:
def rouge_calc(data,i):
    if(i%25 == 0):
        print(i)
    row = data.values[i]
    rouge=Rouge()
    scores = rouge.get_scores(row['respnse'],row['query'])
    d = pd.DataFrame()
    for key,value in scores[0].items():
        for key1,value1 in value.items():
            d[str(key+"_"+key1)] = value1
    return d

In [ ]:
pool = Pool(14)
rouge_res = [pool.apply(rouge_calc, args=(data,i)) for i in range(data.shape[0])]
pd.concat(rouge_res,axis=0).to_csv("Rogue_features.csv",sep=',')

In [ ]:
pool = Pool(14)
jaccard_res = [pool.apply(jaccard_sim, args=(data,i,time.time())) for i in range(data.shape[0])]
pd.DataFrame(jaccard_res).to_csv("Jaccard_feature.csv",sep=',')

0
0.5289084911346436
25
0.46128296852111816
50
0.444840669631958
75
0.46055030822753906
100
0.38983678817749023
125
0.4638516902923584
150
0.4340691566467285
175
0.4601733684539795
200
0.4744534492492676
225
0.379436731338501
250
0.465242862701416
275
0.45467233657836914
300
0.4673476219177246
325
0.47065281867980957
350
0.45823240280151367
375
0.44254302978515625
400
0.4422798156738281
425
0.4300408363342285
450
0.4389934539794922
475
0.44878411293029785
500
0.470200777053833
525
0.45383143424987793
550
0.41132426261901855
575
0.4317629337310791
600
0.45661330223083496
625
0.4381740093231201
650
0.4472332000732422
675
0.4442760944366455
700
0.4462621212005615
725
0.44997262954711914
750
0.4377129077911377
775
0.4328463077545166
800
0.46411681175231934
825
0.375873327255249
850
0.3837730884552002
875
0.43526244163513184
900
0.43985676765441895
925
0.4454226493835449
950
0.3647286891937256
975
0.33194637298583984
1000
0.36368536949157715
1025
0.3682527542114258
1050
0.3554840087890625
1

8450
0.3635108470916748
8475
0.37910962104797363
8500
0.34912776947021484
8525
0.4008646011352539
8550
0.3963630199432373
8575
0.39539098739624023
8600
0.41280341148376465
8625
0.3852198123931885
8650
0.43523430824279785
8675
0.403749942779541
8700
0.4111661911010742
8725
0.39888906478881836
8750
0.39388537406921387
8775
0.4338412284851074
8800
0.44251036643981934
8825
0.44788408279418945
8850
0.44250965118408203
8875
0.45122218132019043
8900
0.3915724754333496
8925
0.40990185737609863
8950
0.3607203960418701
8975
0.36798882484436035
9000
0.3688499927520752
9025
0.3683762550354004
9050
0.4121711254119873
9075
0.4386918544769287
9100
0.444472074508667
9125
0.3932640552520752
9150
0.3580038547515869
9175
0.43749237060546875
9200
0.4005465507507324
9225
0.3655736446380615
9250
0.39182066917419434
9275
0.4061276912689209
9300
0.4059453010559082
9325
0.35950160026550293
9350
0.3921239376068115
9375
0.40671229362487793
9400
0.40657901763916016
9425
0.4700794219970703
9450
0.4387366771697998


16600
0.3904891014099121
16625
0.4112069606781006
16650
0.35549235343933105
16675
0.3356771469116211
16700
0.4136075973510742
16725
0.4041290283203125
16750
0.405933141708374
16775
0.4209556579589844
16800
0.3929314613342285
16825
0.37145209312438965
16850
0.4079132080078125
16875
0.4112274646759033
16900
0.38846445083618164
16925
0.3692002296447754
16950
0.3589637279510498
16975
0.4344480037689209
17000
0.4130673408508301
17025
0.3588297367095947
17050
0.3587205410003662
17075
0.35753417015075684
17100
0.4080066680908203
17125
0.4464104175567627
17150
0.37036943435668945
17175
0.37018775939941406
17200
0.41463565826416016
17225
0.3959023952484131
17250
0.33475279808044434
17275
0.3616664409637451
17300
0.35703420639038086
17325
0.43419837951660156
17350
0.37279510498046875
17375
0.3650510311126709
17400
0.40439724922180176
17425
0.41880106925964355
17450
0.40854787826538086
17475
0.42804646492004395
17500
0.4040207862854004
17525
0.416001558303833
17550
0.3856501579284668
17575
0.3927

24675
0.425858736038208
24700
0.4348423480987549
24725
0.4552876949310303
24750
0.4306161403656006
24775
0.4183464050292969
24800
0.4590456485748291
24825
0.46569299697875977
24850
0.44664525985717773
24875
0.3627805709838867
24900
0.367816686630249
24925
0.34343481063842773
24950
0.36693501472473145
24975
0.3703029155731201
25000
0.35917210578918457
25025
0.3309340476989746
25050
0.37206411361694336
25075
0.362407922744751
25100
0.35767269134521484
25125
0.35606813430786133
25150
0.40563225746154785
25175
0.36933231353759766
25200
0.37957262992858887
25225
0.35544896125793457
25250
0.3638904094696045
25275
0.36757946014404297
25300
0.35735011100769043
25325
0.36636853218078613
25350
0.3596971035003662
25375
0.39336419105529785
25400
0.39380884170532227
25425
0.3670387268066406
25450
0.4018259048461914
25475
0.3867628574371338
25500
0.39810633659362793
25525
0.3948490619659424
25550
0.42343783378601074
25575
0.34018850326538086
25600
0.43587446212768555
25625
0.43540358543395996
25650


32750
0.42970919609069824
32775
0.43177223205566406
32800
0.43423008918762207
32825
0.43280482292175293
32850
0.44675683975219727
32875
0.43331289291381836
32900
0.4490628242492676
32925
0.4531877040863037
32950
0.4615192413330078
32975
0.425976037979126
33000
0.4444711208343506
33025
0.4374692440032959
33050
0.440016508102417
33075
0.4153120517730713
33100
0.4363126754760742
33125
0.4320065975189209
33150
0.46079587936401367
33175
0.49924325942993164
33200
0.39365458488464355
33225
0.44672274589538574
33250
0.43123364448547363
33275
0.4252431392669678
33300
0.41470885276794434
33325
0.4529263973236084
33350
0.4421670436859131
33375
0.44469356536865234
33400
0.4199824333190918
33425
0.4324338436126709
33450
0.4339320659637451
33475
0.42707395553588867
33500
0.44843196868896484
33525
0.43758225440979004
33550
0.35681748390197754
33575
0.37181639671325684
33600
0.45024800300598145
33625
0.4235415458679199
33650
0.3952016830444336
33675
0.4293937683105469
33700
0.4896125793457031
33725
0.

...

56975
0.3546586036682129
57000
0.39716148376464844
57025
0.43574047088623047
57050
0.46111059188842773
57075
0.42996788024902344
57100
0.4331836700439453
57125
0.39694976806640625
57150
0.35866880416870117
57175
0.3663477897644043
57200
0.444840669631958
57225
0.4201335906982422
57250
0.45218610763549805
57275
0.35399556159973145
57300
0.44336938858032227
57325
0.40500640869140625
57350
0.43022823333740234
57375
0.42810511589050293
57400
0.43242907524108887
57425
0.4418337345123291
57450
0.42831969261169434
57475
0.4159376621246338
57500
0.4136695861816406
57525
0.3703286647796631
57550
0.44313907623291016
57575
0.4737730026245117
57600
0.4370560646057129
57625
0.3985466957092285
57650
0.42902207374572754
57675
0.37215542793273926
57700
0.4680025577545166
57725
0.4667661190032959
57750
0.4185760021209717
57775
0.4438292980194092
57800
0.45832157135009766
57825
0.44820070266723633
57850
0.40419864654541016
57875
0.3493351936340332
57900
0.4427003860473633
57925
0.41828203201293945
57950

0.4299638271331787
65050
0.4316713809967041
65075
0.4120779037475586
65100
0.43512868881225586
65125
0.43849682807922363
65150
0.433882474899292
65175
0.428804874420166
65200
0.4295222759246826
65225
0.43424224853515625
65250
0.43131375312805176
65275
0.4462885856628418
65300
0.43777918815612793
65325
0.44385290145874023
65350
0.4175601005554199
65375
0.44765138626098633
65400
0.4431281089782715
65425
0.43946218490600586
65450
0.44762492179870605
65475
0.4310262203216553
65500
0.4428102970123291
65525
0.4382133483886719
65550
0.4347548484802246
65575
0.44978952407836914
65600
0.4348883628845215
65625
0.42757487297058105
65650
0.44239187240600586
65675
0.4330024719238281
65700
0.4424762725830078
65725
0.43767476081848145
65750
0.4357295036315918
65775
0.45798349380493164
65800
0.44149017333984375
65825
0.4448213577270508
65850
0.46381115913391113
65875
0.43506288528442383
65900
0.42133593559265137
65925
0.4474680423736572
65950
0.44403839111328125
65975
0.428760290145874
66000
0.4433958

73100
0.4648575782775879
73125
0.4543452262878418
73150
0.44506120681762695
73175
0.45937514305114746
73200
0.4394710063934326
73225
0.4443857669830322
73250
0.4467155933380127
73275
0.43309807777404785
73300
0.44091224670410156
73325
0.4364027976989746
73350
0.4378623962402344
73375
0.4135615825653076
73400
0.4371154308319092
73425
0.458376407623291
73450
0.4431736469268799
73475
0.4363884925842285
73500
0.4332759380340576
73525
0.4529726505279541
73550
0.4305238723754883
73575
0.4296140670776367
73600
0.4399080276489258
73625
0.46854639053344727
73650
0.4469609260559082
73675
0.43761610984802246
73700
0.44512343406677246
73725
0.42699623107910156
73750
0.4378082752227783
73775
0.42044496536254883
73800
0.4455678462982178
73825
0.44018983840942383
73850
0.44123244285583496
73875
0.44922947883605957
73900
0.44384765625
73925
0.4396250247955322
73950
0.42882561683654785
73975
0.4114799499511719
74000
0.42503905296325684
74025
0.4308304786682129
74050
0.4360463619232178
74075
0.445711851

81200
0.42151403427124023
81225
0.44277477264404297
81250
0.42702817916870117
81275
0.4382944107055664
81300
0.4347825050354004
81325
0.4368739128112793
81350
0.448397159576416
81375
0.44683218002319336
81400
0.4456801414489746
81425
0.43563079833984375
81450
0.43736982345581055
81475
0.4310343265533447
81500
0.4404428005218506
81525
0.43340063095092773
81550
0.4292795658111572
81575
0.4306151866912842
81600
0.4295163154602051
81625
0.4540853500366211
81650
0.45349621772766113
81675
0.4133799076080322
81700
0.43379974365234375
81725
0.43332362174987793
81750
0.4420747756958008
81775
0.44066691398620605
81800
0.4219493865966797
81825
0.3981003761291504
81850
0.46924924850463867
81875
0.4366610050201416
81900
0.43400025367736816
81925
0.435882568359375
81950
0.4358787536621094
81975
0.4377939701080322
82000
0.44322943687438965
82025
0.432842493057251
82050
0.4666910171508789
82075
0.44078660011291504
82100
0.43526744842529297
82125
0.43676304817199707
82150
0.4700014591217041
82175
0.429

89300
0.4277331829071045
89325
0.42722582817077637
89350
0.43787527084350586
89375
0.4276425838470459
89400
0.43009161949157715
89425
0.43034839630126953
89450
0.433474063873291
89475
0.4281795024871826
89500
0.46110033988952637
89525
0.43106889724731445
89550
0.4402937889099121
89575
0.4239819049835205
89600
0.4466104507446289
89625
0.4376206398010254
89650
0.44454145431518555
89675
0.43744802474975586
89700
0.45783233642578125
89725
0.4419515132904053
89750
0.4433126449584961
89775
0.4416682720184326
89800
0.4333455562591553
89825
0.43819165229797363
89850
0.43154191970825195
89875
0.4397733211517334
89900
0.4395272731781006
89925
0.4364938735961914
89950
0.45328736305236816
89975
0.435516357421875
90000
0.430009126663208
90025
0.4284999370574951
90050
0.4358487129211426
90075
0.42603302001953125
90100
0.4284660816192627
90125
0.4324932098388672
90150
0.4229278564453125
90175
0.44529128074645996
90200
0.42838597297668457
90225
0.4272761344909668
90250
0.4252598285675049
90275
0.42821

In [18]:
pool = Pool(14)
cosine_res = [pool.apply(get_cosine_sim, args=(data,i)) for i in range(0,data.shape[0],10)]
p = []
cosine_res1 = [p.extend(x) for x in cosine_res]
pd.DataFrame(p).to_csv("Cosine_feature.csv",sep=',',index=False)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800
9850
9900
9950
10000
10050
10100
10150

70150
70200
70250
70300
70350
70400
70450
70500
70550
70600
70650
70700
70750
70800
70850
70900
70950
71000
71050
71100
71150
71200
71250
71300
71350
71400
71450
71500
71550
71600
71650
71700
71750
71800
71850
71900
71950
72000
72050
72100
72150
72200
72250
72300
72350
72400
72450
72500
72550
72600
72650
72700
72750
72800
72850
72900
72950
73000
73050
73100
73150
73200
73250
73300
73350
73400
73450
73500
73550
73600
73650
73700
73750
73800
73850
73900
73950
74000
74050
74100
74150
74200
74250
74300
74350
74400
74450
74500
74550
74600
74650
74700
74750
74800
74850
74900
74950
75000
75050
75100
75150
75200
75250
75300
75350
75400
75450
75500
75550
75600
75650
75700
75750
75800
75850
75900
75950
76000
76050
76100
76150
76200
76250
76300
76350
76400
76450
76500
76550
76600
76650
76700
76750
76800
76850
76900
76950
77000
77050
77100
77150
77200
77250
77300
77350
77400
77450
77500
77550
77600
77650
77700
77750
77800
77850
77900
77950
78000
78050
78100
78150
78200
78250
78300
78350
78400
7845

132950
133000
133050
133100
133150
133200
133250
133300
133350
133400
133450
133500
133550
133600
133650
133700
133750
133800
133850
133900
133950
134000
134050
134100
134150
134200
134250
134300
134350
134400
134450
134500
134550
134600
134650
134700
134750
134800
134850
134900
134950
135000
135050
135100
135150
135200
135250
135300
135350
135400
135450
135500
135550
135600
135650
135700
135750
135800
135850
135900
135950
136000
136050
136100
136150
136200
136250
136300
136350
136400
136450
136500
136550
136600
136650
136700
136750
136800
136850
136900
136950
137000
137050
137100
137150
137200
137250
137300
137350
137400
137450
137500
137550
137600
137650
137700
137750
137800
137850
137900
137950
138000
138050
138100
138150
138200
138250
138300
138350
138400
138450
138500
138550
138600
138650
138700
138750
138800
138850
138900
138950
139000
139050
139100
139150
139200
139250
139300
139350
139400
139450
139500
139550
139600
139650
139700
139750
139800
139850
139900
139950
140000
140050

191500
191550
191600
191650
191700
191750
191800
191850
191900
191950
192000
192050
192100
192150
192200
192250
192300
192350
192400
192450
192500
192550
192600
192650
192700
192750
192800
192850
192900
192950
193000
193050
193100
193150
193200
193250
193300
193350
193400
193450
193500
193550
193600
193650
193700
193750
193800
193850
193900
193950
194000
194050
194100
194150
194200
194250
194300
194350
194400
194450
194500
194550
194600
194650
194700
194750
194800
194850
194900
194950
195000
195050
195100
195150
195200
195250
195300
195350
195400
195450
195500
195550
195600
195650
195700
195750
195800
195850
195900
195950
196000
196050
196100
196150
196200
196250
196300
196350
196400
196450
196500
196550
196600
196650
196700
196750
196800
196850
196900
196950
197000
197050
197100
197150
197200
197250
197300
197350
197400
197450
197500
197550
197600
197650
197700
197750
197800
197850
197900
197950
198000
198050
198100
198150
198200
198250
198300
198350
198400
198450
198500
198550
198600

Process ForkPoolWorker-105:
Process ForkPoolWorker-108:
Process ForkPoolWorker-101:
Process ForkPoolWorker-99:
Process ForkPoolWorker-110:
Process ForkPoolWorker-107:
Process ForkPoolWorker-102:
Process ForkPoolWorker-109:
Traceback (most recent call last):
Process ForkPoolWorker-106:
Process ForkPoolWorker-104:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-112:
Process ForkPoolWorker-111:
Process ForkPoolWorker-103:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-100:
Traceback (most recent call last):
  File "/opt/anaconda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/anaconda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/anaconda/lib/python3.6/multiprocessing

KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/opt/anaconda/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
  File "/opt/anaconda/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
Traceback (most recent call last):
  File "/opt/anaconda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/anaconda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/opt/anaconda/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/opt/anaconda/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
